In [ ]:
# !pip install deap

In [40]:
from deap import creator, base, tools
from multiprocessing import Pool
from function_opt.functions import rastrigin
from function_opt.ga_scheme import eaMuPlusLambda
from typing import Callable, Optional, Tuple
import numpy as np

In [12]:
creator.create("BaseFitness", base.Fitness, weights=(1.0,))
creator.create("Individual", np.ndarray, fitness=creator.BaseFitness)

In [57]:
def mutation(individual):
    n = len(individual)
    for i in range(n):
        if rnd.random() < n * 0.15:
            individual[i] += rnd.normal(0.0, 0.2)
            individual[i] = np.clip(individual[i], -5, 5)
    return individual,

class Experiment:

    def _factory(self):
        return np.random.random(self.dim) * 10 - 5
    
    def __init__(self, 
                 func: Callable[[np.array], np.array], 
                 dim: int,
                 population: int,
                 max_iter: int,
                 mut_prob: float,
                 cross_prob: float,
                 pool: Pool = Pool(5),
                 engine: base.Toolbox = base.Toolbox(),
                 factory: Optional[Callable[[], np.array]] = None):
        self.population: int = population
        self.max_iter: int = max_iter
        self.mut_prob: float = mut_prob
        self.cross_prob: float = cross_prob
        self.func: Callable[float, float] = func
        self.dim: int = dim
        self.pool: Pool = pool
        self.engine: base.Toolbox = engine
        self.factory: Callable[[], np.array] = factory
        if self.factory is None:
            self.factory = self._factory

        self.engine.register("map", self.pool.map)
        
        self.engine.register("individual", 
            tools.initIterate, 
            creator.Individual, 
            self.factory
        )

        self.engine.register("population", 
            tools.initRepeat,
            list,
            self.engine.individual,
            self.population
        )

        self.engine.register("mate", tools.cxTwoPoint)
    
        self.engine.register("mutate", 
            tools.mutUniformInt, 
            low=-1, 
            up=1, 
            indpb=0.1)

        self.engine.register("select", tools.selTournament, tournsize=3)
        self.engine.register("evaluate", self.func)

    def run(self):
        pop: int = self.engine.population()
        hof: tools.HallOfFame = tools.HallOfFame(3, np.array_equal)
        stats: tools.Statistics = tools.Statistics(lambda x: x.fitness.values[0])
        
        stats.register("avg", np.mean)
        stats.register("std", np.std)
        stats.register("min", np.min)
        stats.register("max", np.max)

        pop, log = eaMuPlusLambda(
            pop, 
            self.engine,
            mu=self.population,
            lambda_= int(self.population * 0.8),
            cxpb=self.cross_prob,
            mutpb=self.mut_prob,
            ngen=self.max_iter,
            stats=stats,
            halloffame=hof,
            verbose=True
        )

        print("Best = {}".format(hof[0]))
        print("Best fit = {}".format(hof[0].fitness.values[0]))
        return log

In [60]:
def obj(x: np.array) -> Tuple[np.array]:
    return (rastrigin(x),)

scene = Experiment(
    func=obj,
    dim=100,
    population=100,
    max_iter=500,
    mut_prob=0.35,
    cross_prob=0.65
)

log = scene.run()

.23116
82 	80    	9.22035	0.0155471 	9.18647	9.25464
83 	80    	9.22697	0.015512  	9.14441	9.25464
84 	80    	9.23262	0.00974457	9.20846	9.25464
85 	80    	9.23822	0.0112914 	9.23116	9.27898
86 	80    	9.24816	0.0150997 	9.23116	9.30428
87 	80    	9.26257	0.0177879 	9.23116	9.30428
88 	80    	9.27593	0.0146823 	9.25464	9.30428
89 	80    	9.28509	0.012865  	9.25464	9.30428
90 	80    	9.29416	0.012391  	9.27898	9.30428
91 	80    	9.30307	0.00995401	9.27898	9.3582 
92 	80    	9.30896	0.0185905 	9.18647	9.3582 
93 	80    	9.31702	0.0247295 	9.20846	9.38713
94 	80    	9.33331	0.0250181 	9.30428	9.38713
95 	80    	9.35056	0.0240527 	9.30428	9.38713
96 	80    	9.37013	0.0167542 	9.33063	9.38713
97 	80    	9.38049	0.012791  	9.33063	9.38713
98 	80    	9.38636	0.0117811 	9.30428	9.41765
99 	80    	9.38971	0.0149012 	9.27898	9.41765
100	80    	9.39663	0.0157756 	9.33063	9.41765
101	80    	9.40513	0.0150087 	9.38713	9.41765
102	80    	9.41398	0.0099165 	9.38713	9.41765
103	80    	9.41734	0.003036